<a href="https://colab.research.google.com/github/mathjams/machine-learning-basics/blob/main/decision_trees.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from public_tests import *
from utils import *

%matplotlib inline

How decision trees work is that we can first use one-hot encoding to describe a certain set of features, make a "tree" splitting on certain features in order to categorize.

The entropy of a node is $$H(p_1) = -p_1 \text{log}_2(p_1) - (1- p_1) \text{log}_2(1- p_1)$$ which measures how homogenous it is (labels of ex poiosonous vs not).

In [ ]:
import math
def compute_entropy(y):
    """
    Computes the entropy for

    Args:
       y (ndarray): Numpy array indicating whether each example at a node is
           edible (`1`) or poisonous (`0`)

    Returns:
        entropy (float): Entropy at that node

    """
    entropy = 0.
    if len(y)==0:
        return entropy
    else:
        p1=sum(y)/len(y)
        if p1==0 or p1==1:
            return entropy
        else:
            entropy=-p1*math.log2(p1)-(1-p1)*math.log2(1-p1)
    return entropy

Now, we can code to split the dataset based on whether it has a certain features (split based on 0 or 1).

In [ ]:
def split_dataset(X, node_indices, feature):
    """
    Splits the data at the given node into
    left and right branches

    Args:
        X (ndarray):             Data matrix of shape(n_samples, n_features)
        node_indices (list):     List containing the active indices. I.e, the samples being considered at this step.
        feature (int):           Index of feature to split on

    Returns:
        left_indices (list):     Indices with feature value == 1
        right_indices (list):    Indices with feature value == 0
    """

    left_indices = []
    right_indices = []
    samples=X.shape[0]
    features=X.shape[1]
    for i in range(len(node_indices)):
        if X[node_indices[i]][feature]==1:
            left_indices.append(node_indices[i])
        else:
            right_indices.append(node_indices[i])
    return left_indices, right_indices

Now, we can define information gain as $$H(p_1^\text{node})- (w^{\text{left}}H(p_1^\text{left}) + w^{\text{right}}H(p_1^\text{right}))$$, where $w^{\text{left}}$ and $w^{\text{right}}$ are the proportions split to the left and the right respectively. This is how much the entropy changes by splitting (and how much more homogenous it becomes).

In [ ]:
def compute_information_gain(X, y, node_indices, feature):

    """
    Compute the information of splitting the node on a given feature

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        feature (int):           Index of feature to split on

    Returns:
        cost (float):        Cost computed

    """
    left_indices, right_indices = split_dataset(X, node_indices, feature)
    X_node, y_node = X[node_indices], y[node_indices]
    X_left, y_left = X[left_indices], y[left_indices]
    X_right, y_right = X[right_indices], y[right_indices]

    information_gain = 0

    information_gain=compute_entropy(y_node)-(len(X_left)/len(X_node)*compute_entropy(y_left)+len(X_right)/len(X_node)*compute_entropy(y_right))

    return information_gain

Now, at each step we want to make it as homogenous as possible. So, we look for the best feature to split on.

In [ ]:
def get_best_split(X, y, node_indices):
    """
    Returns the optimal feature and threshold value
    to split the node data

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.

    Returns:
        best_feature (int):     The index of the best feature to split
    """

    num_features = X.shape[1]

    best_feature = -1

    for i in range(num_features):
        if compute_information_gain(X, y, node_indices, i)>0 and best_feature==-1:
            best_feature=i
        if compute_information_gain(X, y, node_indices, i)>0 and best_feature!=-1 and compute_information_gain(X, y, node_indices, i)>compute_information_gain(X, y, node_indices, best_feature):
            best_feature=i

    return best_feature

Code for the final tree:

In [ ]:
tree = []

def build_tree_recursive(X, y, node_indices, branch_name, max_depth, current_depth):
    """
    Build a tree using the recursive algorithm that split the dataset into 2 subgroups at each node.
    This function just prints the tree.

    Args:
        X (ndarray):            Data matrix of shape(n_samples, n_features)
        y (array like):         list or ndarray with n_samples containing the target variable
        node_indices (ndarray): List containing the active indices. I.e, the samples being considered in this step.
        branch_name (string):   Name of the branch. ['Root', 'Left', 'Right']
        max_depth (int):        Max depth of the resulting tree.
        current_depth (int):    Current depth. Parameter used during recursive call.

    """

    # Maximum depth reached - stop splitting
    if current_depth == max_depth:
        formatting = " "*current_depth + "-"*current_depth
        print(formatting, "%s leaf node with indices" % branch_name, node_indices)
        return

    # Otherwise, get best split and split the data
    # Get the best feature and threshold at this node
    best_feature = get_best_split(X, y, node_indices)

    formatting = "-"*current_depth
    print("%s Depth %d, %s: Split on feature: %d" % (formatting, current_depth, branch_name, best_feature))

    # Split the dataset at the best feature
    left_indices, right_indices = split_dataset(X, node_indices, best_feature)
    tree.append((left_indices, right_indices, best_feature))

    # continue splitting the left and the right child. Increment current depth
    build_tree_recursive(X, y, left_indices, "Left", max_depth, current_depth+1)
    build_tree_recursive(X, y, right_indices, "Right", max_depth, current_depth+1)